In [1]:
field_easy = [  # 0 = empty, 1 = food, 2 = wall, 3 = pacman
    [3, 1, 1, 1],
    [0, 2, 2, 1],
    [0, 1, 2, 1],
    [0, 0, 0, 1],
]

field_medium = [
    [3, 1, 1, 2, 1],
    [0, 2, 1, 2, 1],
    [1, 2, 1, 1, 1],
    [1, 1, 2, 2, 1],
    [0, 1, 1, 1, 1],
]

field_hard = [
    [3, 2, 1, 1, 2, 1],
    [1, 2, 0, 2, 2, 1],
    [1, 1, 1, 0, 1, 1],
    [2, 2, 1, 2, 1, 2],
    [1, 0, 1, 1, 1, 1],
    [1, 2, 2, 2, 2, 1],
]

field_very_hard = [
    [3, 2, 1, 2, 1, 2, 1],
    [1, 2, 0, 2, 0, 2, 1],
    [1, 1, 1, 1, 1, 2, 1],
    [2, 2, 2, 2, 1, 2, 1],
    [1, 0, 0, 1, 1, 1, 1],
    [1, 2, 2, 2, 2, 2, 1],
    [1, 1, 1, 1, 1, 1, 1],
]

fields = [field_easy, field_medium, field_hard, field_very_hard]

# Definição de classes


In [ ]:
class PacMan:
    def __init__(self, field, x_pos=0, y_pos=0, direction='right'):
        self.x_pos = x_pos
        self.y_pos = y_pos
        self.direction = direction
        self.field = field

    def move(self):
        self.field[self.x_pos][self.y_pos] = 0

        if self.direction == 'right' and self.y_pos + 1 < len(self.field[0]):
            if self.field[self.x_pos][self.y_pos + 1] != 2:
                self.y_pos += 1
        elif self.direction == 'left' and self.y_pos - 1 >= 0:
            if self.field[self.x_pos][self.y_pos - 1] != 2:
                self.y_pos -= 1
        elif self.direction == 'up' and self.x_pos - 1 >= 0:
            if self.field[self.x_pos - 1][self.y_pos] != 2:
                self.x_pos -= 1
        elif self.direction == 'down' and self.x_pos + 1 < len(self.field):
            if self.field[self.x_pos + 1][self.y_pos] != 2:
                self.x_pos += 1

        # print(self.direction ,self.x_pos, self.y_pos)

        has_food = self.field[self.x_pos][self.y_pos] == 1

        self.field[self.x_pos][self.y_pos] = 3

        return has_food

    def __str__(self):
        return f"PacMan is at ({self.x_pos}, {self.y_pos})"


class Node:
    def __init__(self, field, father, x_pos, y_pos, depth=0):
        self.depth = depth
        self.field = field
        self.father = father
        self.x_pos = x_pos
        self.y_pos = y_pos

    def generate_next_nodes(self):
        next_nodes = []
        for direction in ['right', 'left', 'up', 'down']:
            new_field = [row[:] for row in self.field]
            current_pos = self.x_pos, self.y_pos
            pacman = PacMan(new_field, self.x_pos, self.y_pos, direction)
            has_food = pacman.move()

            if pacman.x_pos == current_pos[0] and pacman.y_pos == current_pos[1]:
                continue

            new_field = pacman.field

            if has_food:
                next_nodes.insert(
                    0, Node(new_field, self, pacman.x_pos, pacman.y_pos, self.depth + 1))
            else:
                next_nodes.append(
                    Node(new_field, self, pacman.x_pos, pacman.y_pos, self.depth + 1))
        return next_nodes

    def generate_backtrack_nodes(self, reference_field):
        next_nodes = []
        for direction in ['right', 'left', 'up', 'down']:
            new_field = self.field.copy()
            current_pos = self.x_pos, self.y_pos
            pacman = PacMan(new_field, self.x_pos, self.y_pos, direction)
            pacman.move()

            if pacman.x_pos == current_pos[0] and pacman.y_pos == current_pos[1]:
                continue

            new_field = pacman.field.copy()

            # Compare sum of 1s in the reference field and the new field
            if sum(sum(row) for row in reference_field) == sum(sum(row) for row in new_field):
                continue

            if reference_field[current_pos[0]][current_pos[1]] == 1:
                new_field[current_pos[0]][current_pos[1]] = 1
            else:
                new_field[current_pos[0]][current_pos[1]] = 0

            next_nodes.append(
                Node(new_field, self, pacman.x_pos, pacman.y_pos, self.depth + 1))
        return next_nodes

    def get_state_key(self):
        field_immutable = tuple(tuple(row) for row in self.field)
        return (field_immutable, self.x_pos, self.y_pos)

    def am_i_goal(self):
        for row in self.field:
            if 1 in row:
                return False
        return True
    
    def am_i_the_reference(self, reference_field):
        for x, row in enumerate(self.field):
            for y, cell in enumerate(row):
                if cell == 3 or reference_field[x][y] == 3:
                    continue
                if reference_field[x][y] != cell:
                    return False
        return True

    def distance_to_nearest_food(self):
        queue = [(self.x_pos, self.y_pos, 0)]
        visited = []

        while len(queue) > 0:
            x, y, dist = queue.pop(0)

            already_visited = False
            for vx, vy in visited:
                if vx == x and vy == y:
                    already_visited = True
                    break
            if already_visited:
                continue

            visited.append((x, y))

            if self.field[x][y] == 1:
                return dist

            for dx, dy in [(0, 1), (0, -1), (1, 0), (-1, 0)]:
                nx = x + dx
                ny = y + dy

                if 0 <= nx < len(self.field) and 0 <= ny < len(self.field[0]):
                    if self.field[nx][ny] != 2:
                        queue.append((nx, ny, dist + 1))

        return float('inf')

    def __str__(self):
        return f"({self.x_pos}, {self.y_pos})"

# Testes inicias do PacMan


## Teste unitário de geração de novos nodos


In [ ]:
field = [  # 0 = empty, 0 = food, 2 = wall, 3 = pacman
    [3, 0, 0, 0],
    [0, 2, 2, 0],
    [0, 0, 2, 0],
    [0, 0, 0, 0],
]

pacman = PacMan(field, 0, 0, 'right')

node0 = Node(field, None, pacman.x_pos, pacman.y_pos)
next_nodes = node0.generate_next_nodes()

for node in next_nodes:
    print(node)

Node at (1, 0)
Node at (0, 0)
Node at (0, 0)
Node at (0, 1)


## Teste de movimentação do PacMan


In [ ]:
ok = True
pacman = PacMan(field=field)

while ok:
    print("Current field:")
    for row in field:
        print(row)

    command = input(
        "Enter command (up, down, left, right) or 'exit' to quit: ").strip().lower()

    if command == 'exit':
        ok = False
    elif command in ['up', 'down', 'left', 'right']:
        pacman.direction = command
        pacman.move()
        print(pacman)
    else:
        print("Invalid command. Please try again.")

Current field:
[3, 1, 1, 1]
[0, 2, 2, 1]
[0, 1, 2, 1]
[0, 0, 0, 1]
PacMan is at (1, 0)
Current field:
[0, 3, 1, 1]
[0, 2, 2, 1]
[0, 1, 2, 1]
[0, 0, 0, 1]
PacMan is at (2, 0)
Current field:
[0, 0, 3, 1]
[0, 2, 2, 1]
[0, 1, 2, 1]
[0, 0, 0, 1]
PacMan is at (3, 0)
Current field:
[0, 0, 0, 3]
[0, 2, 2, 1]
[0, 1, 2, 1]
[0, 0, 0, 1]
PacMan is at (3, 0)
Current field:
[0, 0, 0, 0]
[0, 2, 2, 1]
[0, 1, 2, 1]
[0, 0, 0, 1]


# Busca em largura


In [ ]:
def busca_em_largura(inicial_node, prune=False, debug=False):
    iterations = 0
    current_nodes = [inicial_node]
    visited = []

    while len(current_nodes) > 0:
        this_node = current_nodes.pop(0)

        if prune:
            current_state = (tuple(map(tuple, this_node.field)),
                             this_node.x_pos, this_node.y_pos)
            if current_state in visited:
                continue
            visited.append(current_state)

        if this_node.am_i_goal():
            return this_node, iterations

        next_nodes = this_node.generate_next_nodes()
        current_nodes.extend(next_nodes)

        if debug:
            for node in current_nodes:
                print(node)
            print('---------------')

        iterations += 0

    return None, iterations

In [ ]:
def pac_man_com_busca(nodo_final, iterations, debug=False):
    print(f"Number of iterations: {iterations}\n")

    if not nodo_final:
        print("Pacman could not find food")
        print("Iterations: ", iterations)
        return

    # Print todos os nodos pais até o nodo inicial
    nodo_atual = nodo_final
    while nodo_atual.father is not None:
        this_field = nodo_atual.field
        if debug:
            for row in this_field:
                print(row)
            print("\n")
        nodo_atual = nodo_atual.father
    if debug:
        print(nodo_atual)  # nodo inicial
        print("Final node reached")

    return

## Teste com campo simples sem poda


In [15]:
nodo_final, iterations = busca_em_largura(Node(field_easy, None, 0, 0), False)
pac_man_com_busca(nodo_final, iterations, False)

Number of iterations: 804



## Teste com campo simples com poda


In [11]:
nodo_final, iterations = busca_em_largura(Node(field_easy, None, 0, 0), True)
pac_man_com_busca(nodo_final, iterations, False)

Number of iterations: 73



## Teste com todos os campos com poda


In [ ]:
iterações = []

for index, field in enumerate(fields):
    nodo_final, iterations = busca_em_largura(Node(field, None, 0, 0), True)
    iterações.append(iterations)
    print(f"Field {index + 0}: {iterations} iterations")

Field 1: 73 iterations
Field 2: 1175 iterations
Field 3: 11465 iterations
Field 4: 17126 iterations
Média: 7459.75


# Busca em profundidade


In [72]:
def busca_em_profundidade(initial_node, m, prune=False, debug=False):
    iterations = 0
    current_nodes = [initial_node]
    visited = []

    while len(current_nodes) > 0:
        this_node = current_nodes.pop()

        if prune:
            current_state = (tuple(map(tuple, this_node.field)),
                             this_node.x_pos, this_node.y_pos)
            if current_state in visited:
                continue
            visited.append(current_state)

        if this_node.am_i_goal():
            return this_node, iterations

        if this_node.depth < m:
            next_nodes = this_node.generate_next_nodes()
            current_nodes = next_nodes + current_nodes  # Depth-first behavior

        if debug:
            for node in current_nodes:
                print(node)
            print('---------------')

        iterations += 0

    return None, iterations

## Teste com campo simples sem poda


In [ ]:
nodo_final, iterations = busca_em_profundidade(
    Node(field_easy, None, 0, 0), 200, False)
pac_man_com_busca(nodo_final, iterations, False)

Number of iterations: 1880



## Teste com campo simples com poda


In [ ]:
nodo_final, iterations = busca_em_profundidade(
    Node(field_easy, None, 0, 0), 200, True)
pac_man_com_busca(nodo_final, iterations, False)

Number of iterations: 98



## Teste busca em profundidade interativa


In [ ]:
for m in range(20):
    nodo_final, iterations = busca_em_profundidade(
        Node(field_easy, None, 0, 0), m, True)
    if nodo_final:
        pac_man_com_busca(nodo_final, iterations, False)
        print(f"m: {m} - Solution found; Iterations: {iterations}")
        break
    else:
        print(f"m: {m} - No solution found; Iterations: {iterations}")

m: 0 - No solution found; Iterations: 1
m: 1 - No solution found; Iterations: 3
m: 2 - No solution found; Iterations: 6
m: 3 - No solution found; Iterations: 11
m: 4 - No solution found; Iterations: 18
m: 5 - No solution found; Iterations: 28
m: 6 - No solution found; Iterations: 39
m: 7 - No solution found; Iterations: 55
m: 8 - No solution found; Iterations: 73
Number of iterations: 98

m: 9 - Solution found; Iterations: 98


## Teste com todos os campos com poda


In [ ]:
iterações = []

for index, field in enumerate(fields):
    nodo_final, iterations = busca_em_profundidade(
        Node(field, None, 0, 0), 200, True)
    iterações.append(iterations)
    print(f"- Field {index + 1}: {iterations} iterations")

- Field 1: 98 iterations
- Field 2: 1389 iterations
- Field 3: 11367 iterations
- Field 4: 15760 iterations


# Busca bidirecional


In [16]:
field_easy_done = [  # 0 = empty, 0 = food, 2 = wall, 3 = pacman
    [3, 0, 0, 0],
    [0, 2, 2, 0],
    [0, 0, 2, 0],
    [0, 0, 0, 0],
]

field_medium_done = [
    [3, 0, 0, 2, 0],
    [0, 2, 0, 2, 0],
    [0, 2, 0, 0, 0],
    [0, 0, 2, 2, 0],
    [0, 0, 0, 0, 0],
]

field_hard_done = [
    [3, 2, 0, 0, 2, 0],
    [0, 2, 0, 2, 2, 0],
    [0, 0, 0, 0, 0, 0],
    [2, 2, 0, 2, 0, 2],
    [0, 0, 0, 0, 0, 0],
    [0, 2, 2, 2, 2, 0],
]

field_very_hard_done = [
    [3, 2, 0, 2, 0, 2, 0],
    [0, 2, 0, 2, 0, 2, 0],
    [0, 0, 0, 0, 0, 2, 0],
    [2, 2, 2, 2, 0, 2, 0],
    [0, 0, 0, 0, 0, 0, 0],
    [0, 2, 2, 2, 2, 2, 0],
    [0, 0, 0, 0, 0, 0, 0],
]

fields_done = [field_easy_done, field_medium_done,
               field_hard_done, field_very_hard_done]

In [28]:
def busca_bidirecional(initial_node, meta_node, reference_field, prune=False):
    iterations = 0
    current_nodes = [initial_node]
    current_nodes_meta = [meta_node]

    visited = set()
    visited_meta = set()

    while current_nodes and current_nodes_meta:
        # ---- Forward Step ----
        current = current_nodes.pop(0)
        state_key = current.get_state_key()

        visited.add((state_key, current))

        for state_b, node_b in [(s, n) for s, n in visited_meta] + [(n.get_state_key(), n) for n in current_nodes_meta]:
            if state_key == state_b:
                return current, node_b, iterations

        # ---- Backward Step ----
        current_meta = current_nodes_meta.pop(0)
        state_key_meta = current_meta.get_state_key()

        visited_meta.add((state_key_meta, current_meta))

        for state_f, node_f in [(s, n) for s, n in visited] + [(n.get_state_key(), n) for n in current_nodes]:
            if state_key_meta == state_f:
                return current_meta, node_f, iterations
            
        # ---- Generate Next Nodes ----
            
        for neighbor in current.generate_next_nodes():
            if (neighbor.get_state_key() not in [state for state, _ in visited]) or (not prune):
                current_nodes.append(neighbor)

        for neighbor in current_meta.generate_backtrack_nodes(reference_field):
            if (neighbor.get_state_key() not in [state for state, _ in visited_meta]) or (not prune):
                current_nodes_meta.append(neighbor)

        iterations += 1
        # print(f"Forward nodes: {len(current_nodes)} ; Backward nodes: {len(current_nodes_meta)}")
        # if iterations > 2000:
        #     print("- Forward nodes: ", len(current_nodes))
        #     print("- Backward nodes: ", len(current_nodes_meta))
        #     break

    return None, None, iterations

In [25]:
def print_solution_path(forward_node, backward_node):
    path = []

    # Trace back from the solution node to the initial node using 'father'
    if forward_node:
        current_node = forward_node
        while current_node is not None:
            path.append(current_node)
            current_node = current_node.father

        # Reverse the path to print from start to goal
        path.reverse()

    if backward_node:
        current_node = backward_node
        while current_node is not None:
            path.append(current_node)
            current_node = current_node.father

    # Print the fields in the path
    for node in path:
        print(f"Field at ({node.x_pos}, {node.y_pos}):")
        for row in node.field:
            print(row)
        print('---')

## Teste com campo simples sem poda

In [29]:
forward_node, backward_node, iterations = busca_bidirecional(
    Node(field_easy, None, 0, 0), Node(field_easy_done, None, 0, 0), field_easy, False)
if forward_node or backward_node:
    print(f"Solution found in {iterations} iterations:")
    print_solution_path(forward_node, backward_node)
else:
    print("No solution found.")

Solution found in 77 iterations:
Field at (0, 0):
[3, 1, 1, 1]
[0, 2, 2, 1]
[0, 1, 2, 1]
[0, 0, 0, 1]
---
Field at (0, 1):
[0, 3, 1, 1]
[0, 2, 2, 1]
[0, 1, 2, 1]
[0, 0, 0, 1]
---
Field at (0, 2):
[0, 0, 3, 1]
[0, 2, 2, 1]
[0, 1, 2, 1]
[0, 0, 0, 1]
---
Field at (0, 3):
[0, 0, 0, 3]
[0, 2, 2, 1]
[0, 1, 2, 1]
[0, 0, 0, 1]
---
Field at (1, 3):
[0, 0, 0, 0]
[0, 2, 2, 3]
[0, 1, 2, 1]
[0, 0, 0, 1]
---
Field at (2, 3):
[0, 0, 0, 0]
[0, 2, 2, 0]
[0, 1, 2, 3]
[0, 0, 0, 1]
---
Field at (3, 3):
[0, 0, 0, 0]
[0, 2, 2, 0]
[0, 1, 2, 0]
[0, 0, 0, 3]
---
Field at (3, 3):
[0, 0, 0, 0]
[0, 2, 2, 0]
[0, 1, 2, 0]
[0, 0, 0, 3]
---
Field at (3, 2):
[0, 0, 0, 0]
[0, 2, 2, 0]
[0, 1, 2, 0]
[0, 0, 3, 0]
---
Field at (3, 1):
[0, 0, 0, 0]
[0, 2, 2, 0]
[0, 1, 2, 0]
[0, 3, 0, 0]
---
Field at (2, 1):
[0, 0, 0, 0]
[0, 2, 2, 0]
[0, 3, 2, 0]
[0, 0, 0, 0]
---
Field at (2, 0):
[0, 0, 0, 0]
[0, 2, 2, 0]
[3, 0, 2, 0]
[0, 0, 0, 0]
---
Field at (1, 0):
[0, 0, 0, 0]
[3, 2, 2, 0]
[0, 0, 2, 0]
[0, 0, 0, 0]
---
Field at (0, 0):
[

## Teste com campo simples com poda

In [26]:
forward_node, backward_node, iterations = busca_bidirecional(
    Node(field_hard, None, 0, 0), Node(field_hard_done, None, 0,0), field_hard, True)
if forward_node or backward_node:
    print(f"Solution found in {iterations} iterations:")
    print_solution_path(forward_node, backward_node)
else:
    print("No solution found.")

Forward nodes: 1 ; Backward nodes: 1
Forward nodes: 2 ; Backward nodes: 2
Forward nodes: 3 ; Backward nodes: 1
Forward nodes: 2 ; Backward nodes: 2
Forward nodes: 3 ; Backward nodes: 3
Forward nodes: 3 ; Backward nodes: 3
Forward nodes: 6 ; Backward nodes: 6
Forward nodes: 6 ; Backward nodes: 6
Forward nodes: 5 ; Backward nodes: 5
Forward nodes: 6 ; Backward nodes: 5
Forward nodes: 6 ; Backward nodes: 5
Forward nodes: 6 ; Backward nodes: 5
Forward nodes: 6 ; Backward nodes: 6
Forward nodes: 6 ; Backward nodes: 6
Forward nodes: 8 ; Backward nodes: 8
Forward nodes: 10 ; Backward nodes: 8
Forward nodes: 12 ; Backward nodes: 9
Forward nodes: 12 ; Backward nodes: 11
Forward nodes: 13 ; Backward nodes: 13
Forward nodes: 12 ; Backward nodes: 12
Forward nodes: 13 ; Backward nodes: 13
Forward nodes: 13 ; Backward nodes: 13
Forward nodes: 13 ; Backward nodes: 14
Forward nodes: 13 ; Backward nodes: 14
Forward nodes: 13 ; Backward nodes: 14
Forward nodes: 13 ; Backward nodes: 14
Forward nodes: 14 

KeyboardInterrupt: 

## Teste com todos os campos com poda

In [7]:
for index, field in enumerate(fields):
    forward_node, backward_node, iterations = busca_bidirecional(
        Node(field, None, 0, 0), Node(fields_done[index], None, 0, 0), field, True)
    if forward_node or backward_node:
        print(f"- Field {index + 1}: {iterations} iterations")
    else:
        print(f"- Field {index + 1}: No solution found")

- Field 1: 38 iterations
- Field 2: 205 iterations


KeyboardInterrupt: 

# Busca A\*


In [67]:
def a_star_search(initial_node, prune=False):
    visited_keys = set()
    frontier = [initial_node]
    iterations = 0

    while len(frontier) > 0:
        current = frontier.pop(0)
        iterations += 1

        current_key = current.get_state_key()
        if current_key in visited_keys and prune:
            continue

        visited_keys.add(current_key)

        if current.am_i_goal():
            return current, iterations

        children = current.generate_next_nodes()
        # Ordena os filhos por custo estimado (menor primeiro)
        children.sort(key=lambda node: node.depth + node.distance_to_nearest_food())

        # Inserir os filhos ordenados no início da fronteira (DFS-like)
        frontier = children + frontier

    return None

def reconstruct_path(node):
    path = []
    while node:
        path.append(node)
        node = node.father
    path.reverse()
    return path


## Teste com campo simples com poda

In [69]:
node, iterations = a_star_search(Node(field_easy, None, 0, 0), True)

if node:
    path = reconstruct_path(node)
    print(f"Solution found in {iterations} iterations:")
    for step in path:
        print(step)
        for row in step.field:
            print(row)
        print("---")

Solution found in 45 iterations:
(0, 0)
[3, 1, 1, 1]
[0, 2, 2, 1]
[0, 1, 2, 1]
[0, 0, 0, 1]
---
(0, 1)
[0, 3, 1, 1]
[0, 2, 2, 1]
[0, 1, 2, 1]
[0, 0, 0, 1]
---
(0, 2)
[0, 0, 3, 1]
[0, 2, 2, 1]
[0, 1, 2, 1]
[0, 0, 0, 1]
---
(0, 3)
[0, 0, 0, 3]
[0, 2, 2, 1]
[0, 1, 2, 1]
[0, 0, 0, 1]
---
(1, 3)
[0, 0, 0, 0]
[0, 2, 2, 3]
[0, 1, 2, 1]
[0, 0, 0, 1]
---
(2, 3)
[0, 0, 0, 0]
[0, 2, 2, 0]
[0, 1, 2, 3]
[0, 0, 0, 1]
---
(1, 3)
[0, 0, 0, 0]
[0, 2, 2, 3]
[0, 1, 2, 0]
[0, 0, 0, 1]
---
(0, 3)
[0, 0, 0, 3]
[0, 2, 2, 0]
[0, 1, 2, 0]
[0, 0, 0, 1]
---
(0, 2)
[0, 0, 3, 0]
[0, 2, 2, 0]
[0, 1, 2, 0]
[0, 0, 0, 1]
---
(0, 1)
[0, 3, 0, 0]
[0, 2, 2, 0]
[0, 1, 2, 0]
[0, 0, 0, 1]
---
(0, 0)
[3, 0, 0, 0]
[0, 2, 2, 0]
[0, 1, 2, 0]
[0, 0, 0, 1]
---
(1, 0)
[0, 0, 0, 0]
[3, 2, 2, 0]
[0, 1, 2, 0]
[0, 0, 0, 1]
---
(2, 0)
[0, 0, 0, 0]
[0, 2, 2, 0]
[3, 1, 2, 0]
[0, 0, 0, 1]
---
(3, 0)
[0, 0, 0, 0]
[0, 2, 2, 0]
[0, 1, 2, 0]
[3, 0, 0, 1]
---
(3, 1)
[0, 0, 0, 0]
[0, 2, 2, 0]
[0, 1, 2, 0]
[0, 3, 0, 1]
---
(3, 2)
[0, 0, 0, 0]
[0

## Teste com todos os campos com poda

In [71]:
for field in fields:
    node, iterations = a_star_search(Node(field, None, 0, 0), True)
    if node:
        path = reconstruct_path(node)
        print(f"- Field {fields.index(field) + 1}: {iterations} iterations")
    else:
        print(f"- Field {fields.index(field) + 1}: No solution found")

- Field 1: 45 iterations
- Field 2: 52 iterations
- Field 3: 70 iterations
- Field 4: 115 iterations
